> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.2`: reinforcement learning exploration demo
This illustrates a use case where we aim to achieve an exploration behavior by generating as many as possible diverse solutions by using a predictive model as the main component.

NOTE: The generated solutions might not be entirely reliable since they could be outside of the applicability domain of the predictive model. Predictive models could score highly compounds that are outside of the applicability domain but this score would be likely inaccurate. This mode would be more reliable if we aslo include `matching_substructure` component with a list of desired core structural patterns/scaffolds. Alternatively this mode can be quite successful in combination with docking or pharmacophore similarity. Such examples will be provided with the next releases. 


## 1. Set up the paths
_Please update the following code block such that it reflects your system's installation and execute it._

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent.v3.2")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_Exploration_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## 2. Setting up the configuration 
In the cells below we will build a nested dictionary object that will be eventually converted to JSON file which in turn will be consumed by `REINVENT`. 
You can find this file in your `output_dir` location.

### A) Declare the run type

In [2]:
# initialize the dictionary
configuration = {
    "version": 3,                          # we are going to use REINVENT's newest release
    "run_type": "reinforcement_learning"   # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "scoring" and "create_model"
}

### B) Sort out the logging details
This includes `result_folder` path where the results will be produced.

Also: `REINVENT` can send custom log messages to a remote location. We have retained this capability in the code. if the `recipient` value differs from `"local"` `REINVENT` will attempt to POST the data to the specified `recipient`. 

In [3]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://0.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                # either to local logging or use a remote REST-interface
    "logging_frequency": 10,             # log every x-th steps
    "logging_path": os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "result_folder": os.path.join(output_dir, "results"),     # will hold the compounds (SMILES) and summaries
    "job_name": "Reinforcement learning demo",                # set an arbitrary job name for identification
    "job_id": "demo"                     # only relevant if "recipient" is set to a specific REST endpoint
}

Create `"parameters"` field

In [4]:
# add the "parameters" block
configuration["parameters"] = {}

### C) Set Diversity Filter
During each step of Reinforcement Learning the compounds scored above `minscore` threshold are kept in memory. The scored smiles are written out to a file in the results folder `scaffold_memory.csv`. In the example here we are not using any filter by setting it to `"IdenticalMurckoScaffold"`. This will help to explore the chemical space since using the diversity filters will stimulate generation of more diverse solutions. The maximum average value of the scoring fuinction will be lower in exploration mode because the Agent is encouraged to search for diverse scaffolds rather than to only optimize the ones that are being found so far. The number of generated compounds should be higher in comparison to the exploitation scenario since the diversity is encouraged. 

In [5]:
# add a "diversity_filter"
configuration["parameters"]["diversity_filter"] =  {
    "name": "IdenticalMurckoScaffold",     # other options are: "IdenticalTopologicalScaffold", 
                                           # "IdenticalMurckoScaffold" and "ScaffoldSimilarity"
                                           # -> use "NoFilter" to disable this feature
    "nbmax": 25,                           # the bin size; penalization will start once this is exceeded
    "minscore": 0.4,                       # the minimum total score to be considered for binning
    "minsimilarity": 0.4                   # the minimum similarity to be placed into the same bin
}

### D) Set Inception
* `smiles` provide here a list of smiles to be incepted 
* `memory_size` the number of smiles allowed in the inception memory
* `sample_size` the number of smiles that can be sampled at each reinforcement learning step from inception memory

In [6]:
# prepare the inception (we do not use it in this example, so "smiles" is an empty list)
configuration["parameters"]["inception"] = {
    "smiles": [],                          # fill in a list of SMILES here that can be used (or leave empty)
    "memory_size": 100,                    # sets how many molecules are to be remembered
    "sample_size": 10                      # how many are to be sampled each epoch from the memory
}

### E) Set the general Reinforcement Learning parameters
* `n_steps` is the amount of Reinforcement Learning steps to perform. Best start with 1000 steps and see if thats enough.
* `agent` is the generative model that undergoes transformation during the Reinforcement Learning run.

We reccomend keeping the other parameters to their default values.

In [7]:
# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["reinforcement_learning"] = {
    "prior": os.path.join(ipynb_path, "models/random.prior.new"), # path to the pre-trained model
    "agent": os.path.join(ipynb_path, "models/random.prior.new"), # path to the pre-trained model
    "n_steps": 125,                        # the number of epochs (steps) to be performed; often 1000
    "sigma": 128,                          # used to calculate the "augmented likelihood", see publication
    "learning_rate": 0.0001,               # sets how strongly the agent is influenced by each epoch
    "batch_size": 128,                     # specifies how many molecules are generated per epoch
    "margin_threshold": 50                 # specify the (positive) margin between agent and prior
}

### F) Define the scoring function
We will use a `custom_product` type. The component types included are:
* `predictive_property` which is the target activity to _Aurora_ kinase represented by the predictive `regression` model. Note that we set the weight of this component to be the highest.
* `qed_score` is the implementation of QED in RDKit. It biases the egenration of  molecules towars more "drug-like" space. Depending on the study case can have beneficial or detrimental effect.
* `custom_alerts` the `"smiles"` field  also can work with SMILES or SMARTS

Note: The model used in this example is a regression model


In [8]:
# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",              # this is our default one (alternative: "custom_sum")
    "parallel": False,                     # sets whether components are to be executed
                                           # in parallel; note, that python uses "False" / "True"
                                           # but the JSON "false" / "true"

    # the "parameters" list holds the individual components
    "parameters": [

    # add component: an activity model
    {
        "component_type": "predictive_property", # this is a scikit-learn model, returning
                                                 # activity values
        "name": "Aurora kinase",                 # arbitrary name for the component
        "weight": 6,                            # the weight ("importance") of the component (default: 1)
        "specific_parameters": {
            "model_path": os.path.join(ipynb_path, "models/Aurora_model.pkl"),   # absolute model path
            "transformation": {
                "transformation_type": "sigmoid",  # see description above
                "high": 9,                         # parameter for sigmoid transformation
                "low": 4,                          # parameter for sigmoid transformation
                "k": 0.25,                         # parameter for sigmoid transformation
            },
            "scikit": "regression",                # model can be "regression" or "classification"
            "descriptor_type": "ecfp_counts",      # sets the input descriptor for this model
            "size": 2048,                          # parameter of descriptor type
            "radius": 3,                           # parameter of descriptor type
            "use_counts": True,                    # parameter of descriptor type
            "use_features": True                   # parameter of descriptor type
        }
    },

    # add component: QED
    {
        "component_type": "qed_score", # this is the QED score as implemented in RDKit
        "name": "QED",        # arbitrary name for the component
        "weight": 2           # the weight ("importance") of the component (default: 1)                      
    },

    # add component: enforce to NOT match a given substructure
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",               # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "specific_parameters": {
            "smiles": [                            # specify the substructures (as list) to penalize
                "[*;r8]",
                "[*;r9]",
                "[*;r10]",
                "[*;r11]",
                "[*;r12]",
                "[*;r13]",
                "[*;r14]",
                "[*;r15]",
                "[*;r16]",
                "[*;r17]",
                "[#8][#8]",
                "[#6;+]",
                "[#16][#16]",
                "[#7;!n][S;!$(S(=O)=O)]",
                "[#7;!n][#7;!n]",
                "C#C",
                "C(=[O,S])[O,S]",
                "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
                "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
                "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
                "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
                "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
                "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
            ]
        }
    }]
}
configuration["parameters"]["scoring_function"] = scoring_function

#### NOTE:  Getting the selectivity score component to reach satisfactory levels is non-trivial and might take considerably higher number of steps

## 3. Write out the configuration

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [9]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "RL_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## 4. Run `REINVENT`
Now it is time to execute `REINVENT` locally. Note, that depending on the number of epochs (steps) and the execution time of the scoring function components, this might take a while. 

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent.v3.2

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [10]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path}

In [11]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

# prepare the output to be parsed
list_epochs = re.findall(r'INFO.*?local', captured_err_stream.stdout, re.DOTALL)
data = [epoch for idx, epoch in enumerate(list_epochs) if idx in [1, 75, 124]]
data = ["\n".join(element.splitlines()[:-1]) for element in data]

Below you see the print-out of the first, one from the middle and the last epoch, respectively. Note, that the fraction of valid `SMILES` is high right from the start (because we use a pre-trained prior). You can see the partial scores for each component for the first couple of compounds, but the most important information is the average score. You can clearly see how it increases over time.

In [12]:
for element in data:
    print(element)

INFO     
 Step 0   Fraction valid SMILES: 98.4   Score: 0.2694   Time elapsed: 0   Time left: 0.0
  Agent     Prior     Target     Score     SMILES
-31.68    -31.68    -31.68      0.00      c1c(Oc2ccc(OC)cc2C(Nc2cccc(C(F)(F)F)c2)=O)cc2c(c1)OCO2
-24.74    -24.74     29.85      0.43      c1(C)ccc(SCC(Nc2ccc(-c3csc(C)n3)cc2)=O)cc1
-38.29    -38.29    -38.29      0.00      c1(COC([N+](=O)[O-])=NNC(c2ccncc2)=O)sccc1C
-21.00    -21.00     31.64      0.41      c1c(NC(=O)C=Cc2ccc(F)cc2)c(C(N)=O)ccc1
-32.55    -32.55     15.23      0.37      c1(C(N2CCN(C)CC2)=O)ccc(C)c2cc3c(cc12)cccn3
-26.17    -26.17     22.14      0.38      c12c(cc(F)cc1)C(=C1CCN(C(C=Cc3cccs3)=O)CC1)c1ncccc1CC2
-36.73    -36.73      6.33      0.34      c1cc2c(cc1)C(=NS(=O)(c1cc(C)c(Cl)cc1Cl)=O)C(=O)N2Cc1ccc2c(n1)cccc2
-27.30    -27.30     28.54      0.44      C1CN(c2cc(N3CCN(C(c4nc5c(cccc5)[nH]4)=O)CC3)ncn2)CC1
-22.68    -22.68    -22.68      0.00      O=C1C=CC(O)(c2n[nH]nn2)O1
-26.01    -26.01     27.14      0.42      N1(Cc

## 5. Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_RL_demo

# make sure, you have activated the proper environment
conda activate reinvent.v3.2

# start tensorboard
tensorboard --logdir progress.log
```

Then copy the link provided to a browser window, e.g. "http://workstation.url.com:6006/". The following figures are exmaple plots - remember, that there is always some randomness involved. In `tensorboard` you can monitor the individual scoring function components. 

The score for predicted Aurora Kinase activity.

![](img/explore_aurora_kinase.png)

The average score over time.

![](img/explore_avg_score.png)

It might also be informative to look at the results from the prior (dark blue), the agent (blue) and the augmented likelihood (purple) over time.

![](img/explore_nll_plot.png)

And last but not least, there is a "Images" tab available that lets you browse through the compounds generated in an easy way. In the molecules, the substructure matches that were defined to be required are highlighted in red (if present). Also, the total scores are given per molecule.

![](img/molecules.png)

The results folder will hold four different files: the agent (pickled), the input JSON (just for reference purposes), the memory (highest scoring compounds in `CSV` format) and the scaffold memory (in `CSV` format).

In [13]:
!head -n 15 {output_dir}/results/memory.csv

,smiles,score,likelihood
126,N(=c1[nH]c(-n2nnc(CCC)c2)nc(=NCC)[nH]1)CC,0.6718567,-29.083855
126,Cc1c[nH]c(=NCCn2cc(CN(C)C)nn2)nc1,0.6282014,-30.314497
114,c1c2c3c(cc1)CC1N(CC=C)CCC34C(C(O)C(CC)CC41)O2,0.61468065,-30.523155
36,c1cc(CCNC(CCN=c2nccc[nH]2)=O)ccc1,0.60359675,-21.289137
81,N(c1c(=O)n(CC(NCc2cc(Cl)c(Cl)cc2)=O)ccc1)C,0.59801006,-22.1726
92,C(C(=O)N)n1cc(-c2ccc(OC)c(OC)c2)cc(NC(C)=O)c1=O,0.5958882,-32.15735
114,Clc1cc(Cl)ccc1Cc1nncc(N2CCNCC2C)c1,0.59467584,-32.407734
89,CCN(c1nc(-c2cc3c([nH]nc3)cc2)cn2ccnc21)CC,0.5946115,-30.064325
70,O=C1CCCN1CCCNC(c1c(O)c2c(c(CC)ccc2)[nH]1)=O,0.5933007,-34.257744
78,c1nccc(CN=c2[nH]nnn2CCCC)c1,0.59266245,-24.547398
62,FC(c1cc(-c2nnn(CCCS(=O)(NC)=O)c2)c[nH]c1=N)(F)F,0.5915846,-31.9637
127,c1(OC)c(O)ccc(-c2c(=NC(CCC)=O)[nH]ccc2)c1,0.5904153,-25.91703
23,[nH]1c(=NCc2ccc(NS(CC)(=O)=O)cc2)ccc(N2CCCC2)n1,0.5896707,-30.422663
109,c1c(C(c2ccno2)Cc2ccc(Cl)cc2)cc(=NC)[nH]c1,0.5894137,-28.653736
